<a href="https://colab.research.google.com/github/MarinaWolters/Coding-Tracker/blob/master/homework5_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5: Advanced Vector Space Models

## Due Date: Jun 19
## Total Points: 116 points + 8 bonus
- **Overview**: In this assignment, we will examine some advanced uses of vector representations of words. We are going to look at three different problems:

  - Solving word relation problems like analogies using word embeddings.
  - Comparing correlation for human judgments of similarity to the vector similarities
  - Discovering the different senses of a ‘polysemous’ word by clustering together its paraphrases.

- **Delieverables:** This assignment has several deliverables:
  - Code (this notebook) *(Automatic Graded)*
    - Part 1: answers to questions
    - Part 3: 4 different clustering functions
  - Write Up (include in this notebook or a separate Report.pdf) *(Manually Graded)*
    - Part 2: answers to questions
    - Part 3: F-scores for clustering algorithms & discussions about your models
  - Leaderboard Without K *(Automatic Graded on GradeScope)*
    - `test_nok_output_leaderboard.txt` = Task 3.4 output file
  - Leaderboard With K *(Automatic Graded on GradeScope)*
    - `test_output_leaderboard.txt` = Task 3.2 or 3.3 output file

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note.


## Recommended Readings
- [Vector Semantics](https://web.stanford.edu/~jurafsky/slp3/6.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft).
- [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf?). Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean. ArXiV 2013.
- [Linguistic Regularities in Continuous Space Word Representations](https://www.aclweb.org/anthology/N13-1090). Tomas Mikolov, Wen-tau Yih, Geoffrey Zweig. NAACL 2013.
- [Discovering Word Senses from Text](https://cs.uwaterloo.ca/~cdimarco/pdf/cs886/Pantel+Lin02.pdf). Patrick Pangel and Dekang Ling. KDD 2002.
- [Linguistic Regularities in Sparse and Explicit Word Representations](https://aclanthology.org/W14-1618.pdf). Patrick Pangel and Dekang Ling. CoNLL 2014.
- [Clustering Paraphrases by Word Sense](https://www.cis.upenn.edu/~ccb/publications/clustering-paraphrases-by-word-sense.pdf). Anne Cocos and Chris Callison-Burch. NAACL 2016.

## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup [4 points]

In [ ]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [ ]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [ ]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 81449864 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW5', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 81449864

Make sure this correct or we will not be able to store your grade


In [ ]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'CIS 5300 Student'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**

In [ ]:
### This cell might take 3 min to run ###
! echo "Installing Magnitude.... (please wait, can take a while)"
! (curl https://raw.githubusercontent.com/plasticityai/magnitude/master/install-colab.sh | /bin/bash 1>/dev/null 2>/dev/null)
! echo "Done installing Magnitude."

Installing Magnitude.... (please wait, can take a while)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   137  100   137    0     0   1078      0 --:--:-- --:--:-- --:--:--  1078
Done installing Magnitude.


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-22 20:53:10.075169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 20:53:10.893416: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 119.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-06-22 20:53:21.036098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operation

In [ ]:
!pip uninstall -y annoy
!pip install annoy

Found existing installation: annoy 1.17.3
Uninstalling annoy-1.17.3:
  Successfully uninstalled annoy-1.17.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=583991 sha256=338bf9867e6aac03ed84b038f85c8d082d2ba80361f273ed44c6262a14f99c73
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
!pip install -U lz4==1.0.0
!pip install -U xxhash==1.0.1
!pip install -U fasteners==0.14.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lz4: filename=lz4-1.0.0-cp310-cp310-linux_x86_64.whl size=913246 sha256=66eeac72dfd053817034ba8f245ca5bd29628245e29e80f1380f44af847b1c3c
  Stored in directory: /root/.cache/pip/wheels/40/dd/7b/fbef788e5fa2c03b81b0a8e443c5f62d4449e75d7838009c0f
Successfully built lz4
  Attempting uninstall: lz4
    Found existing installation: lz4 4.3.2
    Uninstalling lz4-4.3.2:
      Successfully uninstalled lz4-4.3.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for xxhash: filename=xxhash-1.0.1-cp310-cp310-linux_x86_64.whl size=44615 sha256=39a84708588fca5712a8a8e492266fd3459c919d190ac90aa5cbaf3505b7e688
  Stored in directory: /root/.cache/pip/wheels/e3/

In [ ]:
import spacy

In [ ]:
# Check your python version
!python --version

Python 3.10.12


If your python version is >= 3.9, run the code cell below before importing from pymaginitude:

In [ ]:
spacy.load('en_core_web_sm')
import collections
collections.Sequence = collections.abc.Sequence
collections.Mapping = collections.abc.Mapping
collections.MutableMapping = collections.abc.MutableMapping
collections.Iterable = collections.abc.Iterable
collections.MutableSet = collections.abc.MutableSet
collections.Callable = collections.abc.Callable

In [ ]:
from pymagnitude import * # if you encounter an error for this line, try re-running it - I know it's silly but it might work

In [ ]:
# this might take ~2min to run
!wget http://magnitude.plasticity.ai/word2vec/light/GoogleNews-vectors-negative300.magnitude

--2023-06-22 20:55:31--  http://magnitude.plasticity.ai/word2vec/light/GoogleNews-vectors-negative300.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.217.125.245, 52.217.174.197, 52.217.234.77, ...
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.217.125.245|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4211335168 (3.9G) [binary/octet-stream]
Saving to: ‘GoogleNews-vectors-negative300.magnitude’

GoogleNews-vectors- 100%[===================>]   3.92G  39.1MB/s    in 91s     

2023-06-22 20:57:03 (44.2 MB/s) - ‘GoogleNews-vectors-negative300.magnitude’ saved [4211335168/4211335168]



In [ ]:
from itertools import combinations
from prettytable import PrettyTable
from sklearn.cluster import KMeans
import random
import pandas as pd
import numpy as np
import scipy.stats as stats

In [ ]:
# !gdown 1luyNlDu0GdH_B3D6rjYLKIGGfz_S3yU5 # GoogleNews-vectors-negative300.filter.magnitude
!gdown 17a4uC7eNrYdtVlW60wshjyDLcFxTtq0y # SimLex-999.txt
!gdown 1h2DHMuubO7OEVxmGQGbvb2Ovj_A6hakC # dev_input.txt
!gdown 1I83_VA_i_UB-9cf9GcEe5oGPoc8-ZmLh # dev_output.txt
!gdown 1CjK3eYkacyxo3gdLbf9IGdk1DFEfAYvM # test_input.txt
!gdown 1sZuq8a2zHJfe6bLjrK3wD2jrZWkQ0-6S # test_nok_input.txt
!gdown 1gK13ZVDMA5XYi8sZY8G1gOIZMdxGTuay # coocvec-500mostfreq-window-3.vec.filter.magnitude

Downloading...
From: https://drive.google.com/uc?id=17a4uC7eNrYdtVlW60wshjyDLcFxTtq0y
To: /content/SimLex-999.txt
100% 43.0k/43.0k [00:00<00:00, 50.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h2DHMuubO7OEVxmGQGbvb2Ovj_A6hakC
To: /content/dev_input.txt
100% 17.4k/17.4k [00:00<00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I83_VA_i_UB-9cf9GcEe5oGPoc8-ZmLh
To: /content/dev_output.txt
100% 23.1k/23.1k [00:00<00:00, 99.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CjK3eYkacyxo3gdLbf9IGdk1DFEfAYvM
To: /content/test_input.txt
100% 3.81k/3.81k [00:00<00:00, 27.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sZuq8a2zHJfe6bLjrK3wD2jrZWkQ0-6S
To: /content/test_nok_input.txt
100% 4.55k/4.55k [00:00<00:00, 28.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gK13ZVDMA5XYi8sZY8G1gOIZMdxGTuay
To: /content/coocvec-500mostfreq-window-3.filter.magnitude
100% 3.50M/3.50M [00:00<00:00, 177MB/s]


# Section 1: Exploring Analogies and Other Word Pair Relationships [4 points]
**Background:** Word2vec is a very cool word embedding method that was developed by [Thomas Mikolov et al](https://aclanthology.org/N13-1090/). One of the noteworthy things about the method is that it can be used to solve word analogy problems like:

***man is to king as woman is to [blank]***

The way that it they take the vectors representing king, man and woman and perform some vector arithmetic to produce a vector that is close to the expected answer:

***king − man + woman ≈ queen***

We can find the nearest vector in the vocabulary by looking for argmax *cos(x, king − man + woman)*. Omar Levy has an explanation of the method in this [Quora post](https://www.quora.com/unanswered/How-does-Mikolovs-word-analogy-for-word-embedding-work-How-can-I-code-such-a-function) and in the [paper](https://aclanthology.org/W14-1618/).

In addition to solving this sort of analogy problem, the same sort of vector arithmetic was used with word2vec embeddings to find relationships between pairs of words like the following:

<img src='https://drive.google.com/uc?id=1_ewkcJ6EQMuIK0SrgBulzK7LFi8kD9nD'>

In the first part of the assigment, you will play around with the [Magnitude](https://github.com/plasticityai/magnitude) library. You will use Magnitude to load a vector model trained using word2vec, and use it to manipulate and analyze the vectors. In order to proceed further, you need to use the Medium Google-word2vec embedding model trained on Google News by using file `GoogleNews-vectors-negative300.magnitude`. Once the file is downloaded use the following Python commands:

In [ ]:
# file_path = "/content/GoogleNews-vectors-negative300.filter.magnitude"
file_path = "/content/GoogleNews-vectors-negative300.magnitude"
vectors = Magnitude(file_path)

Now you can use vectors to perform queries. For instance, you can query the distance of cat and dog in the following way:

In [ ]:
print(vectors.distance("cat", "dog")) # should be ~0.69

0.69145405


The questions below are designed to familiarize you with the Magnitude word2vec package and get you thinking about what type of semantic information word embeddings can encode. We recommend reading using the [library section](https://github.com/plasticityai/magnitude#using-the-library) to reply to the following set of questions:

- **Problem 1.1:** What is the dimensionality of these word embeddings? Provide an integer answer. [1 point]

In [ ]:
vectors.dim

300

In [ ]:
dimensionality = 300

# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q11_dim', answer = dimensionality) # we only check partial data

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


 - **Problem 1.2:** What are the top-5 most similar words to `picnic` (not including `picnic` itself)? (Hint: try using `vectors.most_similar`) [1 point]

In [ ]:
vectors.most_similar("picnic", topn = 5) # Most similar by key

[('picnics', 0.7400875),
 ('picnic_lunch', 0.7213739),
 ('Picnic', 0.700534),
 ('potluck_picnic', 0.6683274),
 ('picnic_supper', 0.65189123)]

In [ ]:
mostsim = ['picnics', 'picnic_lunch', 'Picnic', 'potluck_picnic', 'picnic_supper']

# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q12_picnic', answer = mostsim) # we only check partial data

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


 - **Problem 1.3:** According to the word embeddings, which of these words is not like the others? `['tissue', 'papyrus', 'manila', 'newsprint', 'parchment', 'gazette']` [1 point]

In [ ]:
vectors.doesnt_match(['tissue', 'papyrus', 'manila', 'newsprint', 'parchment', 'gazette'])

'tissue'

In [ ]:
doesnt_match = "tissue"

# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q13_does_not_match', answer = doesnt_match) # we only check partial data

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


 -  **Problem 1.4:** Solve the following analogy: `leg` is to `jump` as X is to `throw` [1 point]

In [ ]:
vectors.most_similar(positive=['leg', 'throw'], negative=['jump'])

[('forearm', 0.48294652),
 ('shin', 0.47376162),
 ('elbow', 0.4679689),
 ('metacarpal_bone', 0.46781474),
 ('metacarpal_bones', 0.46605822),
 ('ankle', 0.46434426),
 ('shoulder', 0.46183354),
 ('thigh', 0.45393682),
 ('knee', 0.4455708),
 ('ulna_bone', 0.4423491)]

In [ ]:
# TODO
analogy = "forearm"

# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q14_analogy', answer = analogy)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 2: SimLex-999 Dataset Revisited [10 points + 5 Bonus]
Let us revisit [SimLex-999](https://fh295.github.io/simlex.html) dataset from Extra Credit in HW4. We will use `SimLex-999.txt`.

We provided you a script below that:

1. Takes `word1`, `word2`, and `SimLex` columns from the `SimLex-999.txt` dataset,
2. Computes the similarity between `word1` and `word2` using `GoogleNews-vectors-negative300.magnitude` from Part 1
3. Displays correlation for human judgments of similarity to the vector similarities using [Kendall’s Tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient).

In [ ]:
# Reference Code - DO NOT CHANGE
vectors = Magnitude("/content/GoogleNews-vectors-negative300.magnitude")
df = pd.read_csv('/content/SimLex-999.txt', sep='\t')[['word1', 'word2', 'SimLex999']]
human_scores = []
vector_scores = []

counter = 0
for word1, word2, score in df.values.tolist():
    human_scores.append(score)
    similarity_score = vectors.similarity(word1, word2)
    vector_scores.append(similarity_score)
    if counter < 5: # only print the first five
        print(f'{word1},{word2},{score},{similarity_score:.4f}')
        counter += 1

print()
correlation, p_value = stats.kendalltau(human_scores, vector_scores)
print(f'Correlation = {correlation}, P Value = {p_value}')

old,new,1.58,0.2228
smart,intelligent,9.2,0.6495
hard,difficult,8.77,0.6026
happy,cheerful,9.55,0.3838
hard,easy,0.95,0.4710

Correlation = 0.30913428432001067, P Value = 2.6592796177776212e-48


In this part of the assignment we would like for you to explore how the Kendall’s Tau correlation changes based on the similarity. You may use the script we provided or create your own script.

**Please respond to the following questions in your report

Note: **5 Extra points** will be awarded for creativity and a more thorough qualitative analysis.)

 - **Answer 2.1:** What is the least similar 2 pairs of words based on human judgement scores and vector similarity? Do the pairs match? [3 points]

 - **Answer to 2.1**
  * Two the least similar pairs of words, based on Human Judgement are (scale [0..10]):
    * `new - ancient`
    * `shrink - grow`
    * Both of them have the same similarity score of `0.23`
  * Two the least similar pairs of words, based on Vector Similarity are scale [-1.0..1.0]):
    * `house - key` with negative vector similarity score: `-0.041323`
    * `flower - endurance`: `-0.036866`
  * These pairs don't match.
    * However, there is one pair in top-10 least similar pairs that matches:
    * `hole-agreement` with Human Similarity: `0.30`, and Vector Similarity: `0.000394`

In [ ]:
# Adds 'vecScore' column to df
vectors = Magnitude("/content/GoogleNews-vectors-negative300.magnitude")
df = pd.read_csv('/content/SimLex-999.txt', sep='\t')[['word1', 'word2', 'SimLex999']]
human_scores = []
vector_scores = []

for word1, word2, score in df.values.tolist():
    human_scores.append(score)
    similarity_score = vectors.similarity(word1, word2)
    vector_scores.append(similarity_score)

df['vecScore'] = vector_scores
df.head(5)
# print()
# correlation, p_value = stats.kendalltau(human_scores, vector_scores)
# print(f'Correlation = {correlation}, P Value = {p_value}')

,word1,word2,SimLex999,vecScore
0,old,new,1.58,0.222780
1,smart,intelligent,9.20,0.649528
2,hard,difficult,8.77,0.602575
3,happy,cheerful,9.55,0.383774
4,hard,easy,0.95,0.470963


In [ ]:
# Top-10 least similar pairs of words based on Human Judgement score
df.nsmallest(10, 'SimLex999')

,word1,word2,SimLex999,vecScore
75,new,ancient,0.23,0.165659
829,shrink,grow,0.23,0.570728
110,dirty,narrow,0.30,0.113277
724,hole,agreement,0.30,0.000394
735,island,task,0.30,0.035419
742,chapter,tail,0.30,0.101172
759,gun,fur,0.30,0.138761
765,cliff,tail,0.30,0.173206
766,ankle,window,0.30,0.125840
767,princess,island,0.30,0.162527


In [ ]:
# Top-10 least similar pairs of words based on Vector Score
df.nsmallest(10, 'vecScore')

,word1,word2,SimLex999,vecScore
260,house,key,1.90,-0.041323
723,flower,endurance,0.40,-0.036866
756,camera,president,0.48,-0.017754
748,pupil,president,0.78,-0.013141
558,fact,insight,4.77,-0.009103
751,formula,log,1.77,-0.002238
721,jail,choice,1.08,0.000282
724,hole,agreement,0.30,0.000394
990,enter,owe,0.68,0.000408
731,mouse,management,0.48,0.001183


 - **Answer 2.2:** What is the most similar 2 pairs of words based on human judgement scores and vector similarity? Do the pairs match? [3 points]

 - **Answer to 2.2**
  * Two the most similar pairs of words, based on Human Judgement are (scale [0..10]):
    * `vanish - disappear` with Human Similarity score: `9.80`
    * `quick - rapid`: `9.70`
  * Two the most similar pairs of words, based on Vector Similarity are scale [-1.0..1.0]):
    * `south - north` with Vector Similarity score: `0.967454`
    * `north - west`: `0.941540`
  * These pairs don't match.
    * However, the `vanish - disappear` pair comes #3 most sililar pair in Vector Similarity ranking with Human Similarity: `9.80`, and Vector Similarity: `0.900423`

In [ ]:
# Top-5 most similar pairs of words based on Human Judgement score
df.nlargest(5, 'SimLex999')

,word1,word2,SimLex999,vecScore
782,vanish,disappear,9.80,0.900423
22,quick,rapid,9.70,0.497791
205,creator,maker,9.62,0.260506
8,stupid,dumb,9.58,0.817314
16,insane,crazy,9.57,0.733904


In [ ]:
# Top-5 most similar pairs of words based on Vector Score
df.nlargest(5, 'vecScore')

,word1,word2,SimLex999,vecScore
115,south,north,2.20,0.967454
408,north,west,3.63,0.941540
782,vanish,disappear,9.80,0.900423
327,son,father,3.82,0.893309
129,movie,film,8.87,0.867677


- **Answer 2.3:** Provide correlation scores and p values for the following models:
   - (Stanford - GloVe Wikipedia 2014 + Gigaword 5 6B Medium 50D) `glove.6B.50d.magnitude`
   - (Stanford - GloVe Wikipedia 2014 + Gigaword 5 6B Medium 100D)`glove.6B.100d.magnitude`
   - (Stanford - GloVe Wikipedia 2014 + Gigaword 5 6B Medium 200D) `glove.6B.200d.magnitude`
   - (Stanford - GloVe Wikipedia 2014 + Gigaword 5 6B Medium 300D) `glove.6B.300d.magnitude`
   - (Stanford - GloVe Common Crawl Medium 300D) `love.840B.300d.magnitude`

 **How do those correlation value compare to each other?** [4 points]

  **Answer to 2.3**
  * `glove.6B.50d.magnitude`: Correlation = 0.18100126067449063, P Value = 1.2242211264976856e-17
  * `glove.6B.100d.magnitude`: Correlation = 0.20506409092608713, P Value = 3.41228663395174e-22
  * `glove.6B.200d.magnitude`: Correlation = 0.23670323199262908, P Value = 4.9936324557833286e-29
  * `glove.6B.300d.magnitude`: Correlation = 0.25894302181101986, P Value = 2.080389068003349e-34
  * `glove.840B.300d.magnitude`: Correlation = 0.2860664813618063, P Value = 1.293335613361039e-41

  * Analyzing the Correlation between each model vs human similarity we notice that the Correlation increases with every increase in models' dimentions. It doesn't become quite as high as in comparision with GoogleNews vector model `0.31`, but it does get close to it with the Common Crawl & 300 dimentions performing at `0.29`. P Value point to the fact that the results of our correlation measures are statistically significant.

In [ ]:
%%capture
!wget http://magnitude.plasticity.ai/glove/medium/glove.6B.50d.magnitude
!wget http://magnitude.plasticity.ai/glove/medium/glove.6B.100d.magnitude
!wget http://magnitude.plasticity.ai/glove/medium/glove.6B.200d.magnitude
!wget http://magnitude.plasticity.ai/glove/medium/glove.6B.300d.magnitude
!wget http://magnitude.plasticity.ai/glove/medium/glove.840B.300d.magnitude

In [ ]:
# Outputs the Correlation & P Values of each Stanford's model vs Human Silimarity score of SimLex model
vectors_list = ['/content/glove.6B.50d.magnitude', '/content/glove.6B.100d.magnitude',
                '/content/glove.6B.200d.magnitude', '/content/glove.6B.300d.magnitude', '/content/glove.840B.300d.magnitude']
df = pd.read_csv('/content/SimLex-999.txt', sep='\t')[['word1', 'word2', 'SimLex999']]
human_scores = list(df['SimLex999'])

for i in range(5):
  vectors = Magnitude(vectors_list[i])
  vector_scores = []
  for word1, word2, score in df.values.tolist():
      similarity_score = vectors.similarity(word1, word2)
      vector_scores.append(similarity_score)
  correlation, p_value = stats.kendalltau(human_scores, vector_scores)
  print(f'Model = {vectors_list[i]}, Correlation = {correlation}, P Value = {p_value}')

Model = /content/glove.6B.50d.magnitude, Correlation = 0.18100126067449063, P Value = 1.2242211264976856e-17
Model = /content/glove.6B.100d.magnitude, Correlation = 0.20506409092608713, P Value = 3.41228663395174e-22
Model = /content/glove.6B.200d.magnitude, Correlation = 0.23670323199262908, P Value = 4.9936324557833286e-29
Model = /content/glove.6B.300d.magnitude, Correlation = 0.25894302181101986, P Value = 2.080389068003349e-34
Model = /content/glove.840B.300d.magnitude, Correlation = 0.2860664813618063, P Value = 1.293335613361039e-41


# Section 3: Creating Word Sense Clusters [96 points]
**Background:** Many Natural Language Processing (NLP) tasks require knowing the sense of polysemous words, which are words with multiple meanings. For example, the word bug can mean:

1. A creepy crawly thing
2. An error in your computer code
3. A virus or bacteria that makes you sick
4. A listening device planted by the FBI

In past research my PhD students and I have looked into automatically deriving the different meaning of polysemous words like bug by clustering their paraphrases. We have developed a resource called [the paraphrase database (PPDB)](http://paraphrase.org/) that contains of paraphrases for tens of millions words and phrases. For the target word bug, we have an unordered list of paraphrases including: insect, glitch, beetle, error, microbe, wire, cockroach, malfunction, microphone, mosquito, virus, tracker, pest, informer, snitch, parasite, bacterium, fault, mistake, failure and many others. We used automatic clustering group those into sets like:

<img src='https://drive.google.com/uc?id=1-YbbvZ0qwRKPiHZ1ZWOm62dfCkfu4tLJ'>

The clusters in the image above approximate the different word senses of bug, where the 4 circles are the 4 senses of bug. The input to this problem is all the paraphrases in a single list, and the task is to separate them correctly. As humans, this is pretty intuitive, but computers are not that smart. You will explore the main idea underlying our word sense clustering method: which measure the similarity between each pair of paraphrases for a target word and then group together the paraphrases that are most similar to each other. This affinity matrix gives an example of one of the methods for measuring similarity that we tried in our [paper](https://www.cis.upenn.edu/~ccb/publications/clustering-paraphrases-by-word-sense.pdf):

<img src='https://drive.google.com/uc?id=1v1dBzwoSM3S3Y1wDUwqcVBEZ7GxxKKJ4'>

Here the darkness values give an indication of how similar paraphrases are to each other. For instance in this example similarity between *insect* and *pest* is greater than the similarity between insect and error. You can read more about this task in [these](https://www.cis.upenn.edu/~ccb/publications/clustering-paraphrases-by-word-sense.pdf) [papers](https://cs.uwaterloo.ca/~cdimarco/pdf/cs886/Pantel+Lin02.pdf).

In this assignment, we will use vector representations in order to measure their similarities of pairs of paraphrases. You will play with different vector space representations of words to create clusters of word senses. We expect that you have read Jurafsky and Martin [Chapter 6](https://web.stanford.edu/~jurafsky/slp3/6.pdf). Word vectors, also known as word embeddings, can be thought of simply as points in some high-dimensional space. Remember in geometry class when you learned about the Euclidean plane, and 2-dimensional points in that plane? It’s not hard to understand distance between those points – you can even measure it with a ruler. Then you learned about 3-dimensional points, and how to calculate the distance between these. These 3-dimensional points can be thought of as positions in physical space.

Now, do your best to stop thinking about physical space, and generalize this idea in your mind: you can calculate a distance between 2-dimensional and 3-dimensional points, now imagine a point with `N` dimensions. The dimensions don’t necessarily have meaning in the same way as the X,Y, and Z dimensions in physical space, but we can calculate distances all the same.

This is how we will use word vectors in this assignment: as points in some high-dimensional space, where distances between points are meaningful. The interpretation of distance between word vectors depends entirely on how they were made, but for our purposes, we will consider distance to measure semantic similarity. Word vectors that are close together should have meanings that are similar.

With this framework, we can see how to solve our paraphrase clustering problem.

**The Data:**
The input data to be used for this assignment consists of sets of paraphrases corresponding to one of polysemous target words, e.g.

Target	  | Paraphrase set
----------|------------------
note.v    | comment mark tell observe state notice say remark mention
hot.a     | raging spicy blistering red-hot live

(Here the `.v` following the target `note` indicates the part of speech)

Your objective is to automatically cluster each paraphrase set such that each cluster contains words pertaining to a single sense, or meaning, of the target word. Note that a single word from the paraphrase set might belong to one or more clusters.

**Development Data:** The development data consists of two files:

1. words file (input)
2. clusters file (output)

The words file `dev_input.txt` is formatted such that each line contains one target, its paraphrase set, and the number of ground truth clusters `k`, separated by a `::` symbol. You can use `k` as input to your clustering algorithm.

`target.pos :: k :: paraphrase1 paraphrase2 paraphrase3 ...`

The clusters file `dev_output.txt` contains the ground truth clusters for each target word’s paraphrase set, split over k lines:

```
target.pos :: 1 :: paraphrase2 paraphrase6
target.pos :: 2 :: paraphrase3 paraphrase4 paraphrase5
    .
    .
    .
target.pos :: k :: paraphrase1 paraphrase9
```

**Test data:** For testing Tasks 3.1 – 3.3, you will receive only words file `test_input.txt` containing the test target words, number of ground truth clusters and their paraphrase sets. For testing Task 3.4, you will receive only words file `test_nok_input.txt` containing the test target words and their paraphrases sets. Neither order of senses, nor order of words in a cluster matter.

**Evaluation:** There are many possible ways to evaluate clustering solutions. For this homework we will rely on the paired F-score, which you can read more about in [this paper](https://www.cs.york.ac.uk/semeval2010_WSI/paper/semevaltask14.pdf).

The general idea behind paired F-score is to treat clustering prediction like a classification problem; given a target word and its paraphrase set, we call a *positive instance* any pair of paraphrases that appear together in a ground-truth cluster. Once we predict a clustering solution for the paraphrase set, we similarly generate the set of word pairs such that both words in the pair appear in the same predicted cluster. We can then evaluate our set of predicted pairs against the ground truth pairs using precision, recall, and F-score.

V-Measure is another metric that is used to evaluate clustering solutions, however we will not be using it in this Assignment.

**Tasks:**
Your task is to fill in 4 functions: `cluster_random`, `cluster_with_sparse_representation`, `cluster_with_dense_representation`, `cluster_with_no_k`.

We provided 5 utility functions for you to use:

1. `load_input_file(file_path)` that converts the input data (the words file) into 2 dictionaries. The first dictionary is a mapping between a target word and a list of paraphrases. The second dictionary is a mapping between a target word and a number of clusters for a given target word.

2. `load_output_file(file_path)` that converts the output data (the clusters file) into a dictionary, where a key is a target word and a value is it’s list of list of paraphrases. Each list of paraphrases is a cluster. Remember that Neither order of senses, nor order of words in a cluster matter.

3. `get_paired_f_score(gold_clustering, predicted_clustering)` that calculates paired F-score given a gold and predicted clustering for a target word.

4. `evaluate_clusterings(gold_clusterings, predicted_clusterings)` that calculates paired F-score for all target words present in the data and prints the final F-Score weighted by the number of senses that a target word has.

5. `write_to_output_file(file_path, clusterings)` that writes the result of the clustering for each target word into the output file (clusters file)
Full points will be awarded for each of the tasks if your implementation gets above a certain threshold on the test dataset. Please submit to autograder to see thresholds. Note that thresholds are based on the scores from the previous year and might be lowered depending on the average performance.

In [80]:
# Helper functions, DO NOT MODIFY
def load_input_file(file_path):
    """
    Loads the input file to two dictionaries
    :param file_path: path to an input file
    :return: 2 dictionaries:
    1. Dictionary, where key is a target word and value is a list of paraphrases
    2. Dictionary, where key is a target word and value is a number of clusters
    """
    word_to_paraphrases_dict = {}
    word_to_k_dict = {}

    with open(file_path, 'r') as fin:
        for line in fin:
            target_word, k, paraphrases = line.split(' :: ')
            word_to_k_dict[target_word] = int(k)
            word_to_paraphrases_dict[target_word] = paraphrases.split()

    return word_to_paraphrases_dict, word_to_k_dict

def load_output_file(file_path):
    """
    :param file_path: path to an output file
    :return: A dictionary, where key is a target word and value is a list of list of paraphrases
    """
    clusterings = {}

    with open(file_path, 'r') as fin:
        for line in fin:
            target_word, _, paraphrases_in_cluster = line.strip().split(' :: ')
            paraphrases_list = paraphrases_in_cluster.strip().split()
            if target_word not in clusterings:
                clusterings[target_word] = []
            clusterings[target_word].append(paraphrases_list)

    return clusterings

def get_paired_f_score(gold_clustering, predicted_clustering):
    """
    :param gold_clustering: gold list of list of paraphrases
    :param predicted_clustering: predicted list of list of paraphrases
    :return: Paired F-Score
    """
    gold_pairs = set()
    for gold_cluster in gold_clustering:
        for pair in combinations(gold_cluster, 2):
            gold_pairs.add(tuple(sorted(pair)))

    predicted_pairs = set()
    for predicted_cluster in predicted_clustering:
        for pair in combinations(predicted_cluster, 2):
            predicted_pairs.add(tuple(sorted(pair)))

    overlapping_pairs = gold_pairs & predicted_pairs

    precision = 1. if len(predicted_pairs) == 0 else float(len(overlapping_pairs)) / len(predicted_pairs)
    recall = 1. if len(gold_pairs) == 0 else float(len(overlapping_pairs)) / len(gold_pairs)
    paired_f_score = 0. if precision + recall == 0 else 2 * precision * recall / (precision + recall)

    return paired_f_score

def evaluate_clusterings(gold_clusterings, predicted_clusterings):
    """
    Displays evaluation scores between gold and predicted clusterings
    :param gold_clusterings: dictionary where key is a target word and value is a list of list of paraphrases
    :param predicted_clusterings: dictionary where key is a target word and value is a list of list of paraphrases
    :return: N/A
    """
    target_words = set(gold_clusterings.keys()) & set(predicted_clusterings.keys())

    if len(target_words) == 0:
        print('No overlapping target words in ground-truth and predicted files')
        return None

    paired_f_scores = np.zeros((len(target_words)))
    ks = np.zeros((len(target_words)))

    table = PrettyTable(['Target', 'k', 'Paired F-Score'])
    for i, target_word in enumerate(target_words):
        paired_f_score = get_paired_f_score(gold_clusterings[target_word], predicted_clusterings[target_word])
        k = len(gold_clusterings[target_word])
        paired_f_scores[i] = paired_f_score
        ks[i] = k
        table.add_row([target_word, k, f'{paired_f_score:0.4f}'])

    average_f_score = np.average(paired_f_scores, weights=ks)
    print(table)
    print(f'=> Average Paired F-Score:  {average_f_score:.4f}')

def write_to_output_file(file_path, clusterings):
    """
    Writes the result of clusterings into an output file
    :param file_path: path to an output file
    :param clusterings:  A dictionary, where key is a target word and value is a list of list of paraphrases
    :return: N/A
    """
    with open(file_path, 'w') as fout:
        for target_word, clustering in clusterings.items():
            for i, cluster in enumerate(clustering):
                fout.write(f'{target_word} :: {i + 1} :: {" ".join(cluster)}\n')
        fout.close()

## 3.1. Cluster Randomly [11 points]
Write a function `cluster_random(word_to_paraphrases_dict, word_to_k_dict)` that accepts 2 dictionaries:

1. word_to_paraphrases_dict = a mapping between a target word and a list of paraphrases

2. word_to_k_dict = a mapping between a target word and a number of clusters for a given target

The function outputs a dictionary, where the key is a target word and a value is a list of list of paraphrases, where a list of paraphrases represents a distinct sense of a target word.

For this task put paraphrases into distinct senses at random. That is, assign to pick a random word for each cluster, as opposed to picking a random cluster for each word. This will ensure that all clusters have at lease one word in them. We recommend using random packages. Please use 123 as a random seed. Your output should look similar to this on the development dataset:

```
word_to_paraphrases_dict, word_to_k_dict = load_input_file('dev_input.txt')
gold_clusterings = load_output_file('dev_output.txt')
predicted_clusterings = cluster_random(word_to_paraphrases_dict, word_to_k_dict)
evaluate_clusterings(gold_clusterings, predicted_clusterings)
```
```
+----------------+----+----------------+
|     Target     | k  | Paired F-Score |
+----------------+----+----------------+
|    paper.n     | 7  |     0.2978     |
|     play.v     | 34 |     0.0896     |
|     miss.v     | 8  |     0.2376     |
|   produce.v    | 7  |     0.2335     |
|    party.n     | 5  |     0.2480     |
|     note.v     | 3  |     0.6667     |
|     bank.n     | 9  |     0.1515     |
    .
    .
    .
|     eat.v      | 6  |     0.2908     |
|    climb.v     | 6  |     0.2427     |
|    degree.n    | 7  |     0.2891     |
|   interest.n   | 5  |     0.2093     |
+----------------+----+----------------+
=> Average Paired F-Score:  0.2318
```

- **Problem 3.1:** Implement `cluster_random` function. **The augograder for 3.2, 3.3, 3.4 will grade your implementation based on the test-set `f_score` achieved by the clustering.**  [10 points]


In [82]:
def cluster_random(word_to_paraphrases_dict, word_to_k_dict):
    """ Clusters paraphrases randomly.
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :param word_to_k_dict: dictionary, where key is a target word and value is a number of clusters
    :return: dictionary, where key is a target word and value is a list of list of paraphrases, where each list corresponds to a cluster
    """
    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        k = word_to_k_dict[target_word]

        paraph_list_of_lists = []
        random.shuffle(paraphrase_list)
        index_low = 0
        window = len(paraphrase_list) // k
        low_rand_index = 1
        high_rand_index = low_rand_index + window
        div = len(paraphrase_list) - 1

        for i in range(k - 1):
            div = random.randint(low_rand_index, high_rand_index - 1)   #(1,4), (5, 8), (9, 12)
            l = paraphrase_list[index_low:div]                          #[0: 4], [4: 6], [6: 10]
            paraph_list_of_lists.append(l)
            index_low = div
            low_rand_index = high_rand_index    #5, 9, 13
            high_rand_index += window           #9, 13, 17
        l = paraphrase_list[div:]
        paraph_list_of_lists.append(l)

        clusterings[target_word] = paraph_list_of_lists
    return clusterings

- **Answer 3.1:** Run clustering on `dev` data, report the `f_scores` from the `dev` data [1 point]

**Answer to 3.1**
Reporting f_scores from the dev data

*   Average Paired F-Score from `dev` data:  0.1891

```
+----------------+----+----------------+
|     Target     | k  | Paired F-Score |
+----------------+----+----------------+
|    smell.v     | 4  |     0.2432     |
|    image.n     | 9  |     0.1760     |
|   express.v    | 7  |     0.2036     |
|     talk.v     | 6  |     0.2460     |
| performance.n  | 5  |     0.2373     |
|    begin.v     | 8  |     0.1975     |
|     use.v      | 6  |     0.2516     |
|  atmosphere.n  | 6  |     0.2162     |
|     rule.v     | 7  |     0.1909     |
|     bank.n     | 9  |     0.0563     |
|     win.v      | 4  |     0.3655     |
|     wash.v     | 13 |     0.1190     |
|    degree.n    | 7  |     0.2119     |
|   provide.v    | 7  |     0.2368     |
|   receive.v    | 13 |     0.0864     |
|    watch.v     | 5  |     0.2182     |
|     plan.n     | 3  |     0.3989     |
|  different.a   | 1  |     0.0000     |
|   produce.v    | 7  |     0.2011     |
|    treat.v     | 8  |     0.1699     |
|     hear.v     | 5  |     0.2344     |
|   operate.v    | 7  |     0.2018     |
|    simple.a    | 5  |     0.4545     |
|   interest.n   | 5  |     0.2246     |
|   judgment.n   | 7  |     0.2464     |
|     eat.v      | 6  |     0.2219     |
|     mean.v     | 6  |     0.2500     |
|     play.v     | 34 |     0.0541     |
|    party.n     | 5  |     0.2357     |
|    climb.v     | 6  |     0.2824     |
|    source.n    | 9  |     0.1274     |
|   suspend.v    | 6  |     0.1176     |
|    expect.v    | 6  |     0.2902     |
| organization.n | 7  |     0.1896     |
|   shelter.n    | 5  |     0.3773     |
|     miss.v     | 8  |     0.1702     |
|    write.v     | 9  |     0.1306     |
|    paper.n     | 7  |     0.2282     |
|  difference.n  | 5  |     0.2617     |
|     note.v     | 3  |     0.4615     |
+----------------+----+----------------+
=> Average Paired F-Score:  0.1891
```

In [83]:
### Reference Code ###
###### You can use the following code to test your clustering on dev data ######
word_to_paraphrases_dict_dev, word_to_k_dict_dev = load_input_file('dev_input.txt')
predicted_clusterings_random_dev = cluster_random(word_to_paraphrases_dict_dev, word_to_k_dict_dev)
gold_clusterings_dev = load_output_file('dev_output.txt')
f_score = evaluate_clusterings(gold_clusterings_dev, predicted_clusterings_random_dev)
f_score

+----------------+----+----------------+
|     Target     | k  | Paired F-Score |
+----------------+----+----------------+
|    smell.v     | 4  |     0.2432     |
|    image.n     | 9  |     0.1760     |
|   express.v    | 7  |     0.2036     |
|     talk.v     | 6  |     0.2460     |
| performance.n  | 5  |     0.2373     |
|    begin.v     | 8  |     0.1975     |
|     use.v      | 6  |     0.2516     |
|  atmosphere.n  | 6  |     0.2162     |
|     rule.v     | 7  |     0.1909     |
|     bank.n     | 9  |     0.0563     |
|     win.v      | 4  |     0.3655     |
|     wash.v     | 13 |     0.1190     |
|    degree.n    | 7  |     0.2119     |
|   provide.v    | 7  |     0.2368     |
|   receive.v    | 13 |     0.0864     |
|    watch.v     | 5  |     0.2182     |
|     plan.n     | 3  |     0.3989     |
|  different.a   | 1  |     0.0000     |
|   produce.v    | 7  |     0.2011     |
|    treat.v     | 8  |     0.1699     |
|     hear.v     | 5  |     0.2344     |
|   operate.v   

Run the following command to generate the output file for the predicted clusterings for the test dataset.

In [84]:
word_to_paraphrases_dict, word_to_k_dict = load_input_file('test_input.txt')
predicted_clusterings_random = cluster_random(word_to_paraphrases_dict, word_to_k_dict)
write_to_output_file('test_output_random.txt', predicted_clusterings_random)

In [85]:
# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q3_clusters_random', answer = (predicted_clusterings_random, 'random'))

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 3.2 Cluster with Sparse Representations [26 points]

Write a function `cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict)`. The input and output remains the same as in Task 1, however the clustering of paraphrases will no longer be random and is based on sparse vector representation.

We will feature-based (not dense) vector space representation. In this type of VSM, each dimension of the vector space corresponds to a specific feature, such as a context word (see, for example, the term-context matrix described in [Chapter 6.1.2 of Jurafsky & Martin](https://web.stanford.edu/~jurafsky/slp3/6.pdf)). You will calculate cooccurrence vectors on the Reuters RCV1 corpus. It can take a long time to build cooccurrence vectors, so we have pre-built set called `coocvec-500mostfreq-window-3.vec.filter.magnitude`. To save on space, these include only the words used in the given files. This representation of words uses a term-context matrix `M` of size `|V| x D`, where `|V|` is the size of the vocabulary and D=500. Each feature corresponds to one of the top 500 most-frequent words in the corpus. The value of matrix entry `M[i][j]` gives the number of times the context word represented by column `j` appeared within W=3 words to the left or right of the word represented by row `i` in the corpus.

Use one of the clustering algorithms, for instance K-means clustering in `cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict)`. Here is an example of the K-means clustering code:



```
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=k).fit(X)
print(kmeans.labels_)
```



- **Problem 3.2:** Implement `cluster_with_sparse_representation` function [20 points]

In [ ]:
def cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict):
    """
    Clusters paraphrases using sparse vector representation
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :param word_to_k_dict: dictionary, where key is a target word and value is a number of clusters
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    # Note: any vector representation should be in the same directory as this file
    vectors = Magnitude("coocvec-500mostfreq-window-3.filter.magnitude")
    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        k = word_to_k_dict[target_word]
        # TODO: Implement
        clusterings[target_word] = None

    return clusterings

**Answer 3.2.1:** Run clustering on `dev` data, report the `f_scores` from the `dev` data [1 point]

**TODO**: [Report f_scores from the dev data]

In [ ]:
# YOUR CODE HERE (you can re-use reference code from 3.1)


Run the following command to generate the output file for the predicted clusterings for the test dataset.

In [ ]:
word_to_paraphrases_dict, word_to_k_dict = load_input_file('test_input.txt')
predicted_clusterings_sparse = cluster_with_sparse_representation(word_to_paraphrases_dict, word_to_k_dict)
# write_to_output_file('test_output_sparse.txt', predicted_clusterings_sparse)

In [ ]:
# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q3_clusters_sparse', answer = (predicted_clusterings_sparse, 'sparse'))

**Answer 3.2.2:** Provide a brief description of your method in the report, making sure to describe the vector space model you chose, the clustering algorithm you used, and the results of any preliminary experiments you might have run on the dev set.  [5 points]

Suggestions to improve the performance of your model:

 - What if you reduce or increase `D` in the baseline implementation?
 - Does it help to change the window `W` used to extract contexts?
 - Play around with the feature weighting – instead of raw counts, would it help to use PPMI?
 - Try a different clustering algorithm that’s included with the [scikit-learn clustering package](http://scikit-learn.org/stable/modules/clustering.html), or implement your own.
 - What if you include additional types of features, like paraphrases in the [Paraphrase Database](http://www.paraphrase.org/) or the part-of-speech of context words?

The only feature types that are off-limits are WordNet features.

**TODO**: Description of your mothod

## 3.3 Cluster with Dense Representations [28 points]

Write a function `cluster_with_dense_representation(word_to_paraphrases_dict, word_to_k_dict)`. The input and output remains the same as in Task 1 and 2, however the clustering of paraphrases is based on dense vector representation.

We would like to see if dense word embeddings are better for clustering the words in our test set. Run the word clustering task again, but this time use a dense word representation.

For this task, we have also included a file called `GoogleNews-vectors-negative300.filter.magnitude`, which is filtered to contain only the words in the dev/test splits.

As before, use the provided word vectors to represent words and perform one of the clusterings. Here are some suggestions to improve the performance of your model:

 - Try downloading a different dense vector space model from the web, like [Paragram](http://www.cs.cmu.edu/~jwieting/) or [fastText](https://fasttext.cc/docs/en/english-vectors.html).
 - Train your own word vectors, either on the provided corpus or something you find online. You can try the skip-gram, CBOW models, or [GLOVE](https://nlp.stanford.edu/projects/glove/). Try experimenting with the dimensionality.
 - [Retrofitting](https://www.cs.cmu.edu/~hovy/papers/15HLT-retrofitting-word-vectors.pdf) is a simple way to add additional semantic knowledge to pre-trained vectors. The retrofitting code is available here. Experiment with different lexicons, or even try [counter-fitting](http://www.aclweb.org/anthology/N16-1018).

- **Problem 3.3:** Implement `cluster_with_dense_representation` function [20 points]

In [ ]:
def cluster_with_dense_representation(word_to_paraphrases_dict, word_to_k_dict):
    """
    Clusters paraphrases using dense vector representation
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :param word_to_k_dict: dictionary, where key is a target word and value is a number of clusters
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    # Note: any vector representation should be in the same directory as this file
    vectors = Magnitude("GoogleNews-vectors-negative300.magnitude")
    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        k = word_to_k_dict[target_word]
        # TODO: Implement
        clusterings[target_word] = None

    return clusterings

**Answer 3.3.1:** Run clustering on `dev` data, report the `f_scores` from the `dev` data [1 point]

**TODO**: [Report f_scores from the dev data]

In [ ]:
# YOUR CODE HERE (you can re-use reference code from 3.1)

As before, run the following command to generate the output file for the predicted clusterings for the test dataset.

In [ ]:
word_to_paraphrases_dict, word_to_k_dict = load_input_file('test_input.txt')
predicted_clusterings_dense = cluster_with_dense_representation(word_to_paraphrases_dict, word_to_k_dict)
# write_to_output_file('test_output_dense.txt', predicted_clusterings_dense)

In [ ]:
# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q3_clusters_dense', answer = (predicted_clusterings_dense, 'dense'))

**Answer 3.3.2:** Provide a brief description of your method in the report that includes the vectors you used, and any experimental results you have from running your model on the dev set.  [5 points]

**TODO**: [Describe your method]

**Answer 3.3.3:** In addition, for Task 3.2 and 3.3, do an analysis of different errors made by each system – i.e. look at instances that the word-context matrix representation gets wrong and dense gets right, and vice versa, and see if there are any interesting patterns. There is no right answer for this. [2 points]

**TODO**: [Error analysis]

## 3.4 Cluster without K [31 points]

So far we made the clustering problem deliberately easier by providing you with `k`, the number of clusters, as an input. But in most clustering situations the best `k` is not given. To take this assignment one step further, see if you can come up with a way to automatically choose `k`.

Write a function `cluster_with_no_k(word_to_paraphrases_dict)` that accepts only the first dictionary as an input and produces clusterings for given target words.

We have provided an additional test set `test_nok_input.txt`, where the `k` field has been zeroed out. See if you can come up with a method that clusters words by sense, and chooses the best `k` on its own. You can start by assigning `k=5` for all target words as a baseline model.

You might want to try and use the development data to analyze how got is your model in determining `k`.

One of the ways to approach this challenge is to try and select best `k` for a target word and a list of paraphrases is to use try out a range of `k`'s and judge the performance of the clusterings based on some metric, for instance a [silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).

Be sure to describe your method in the Report.

- **Problem 3.4:** Implement `cluster_with_no_k` function [25 points]


In [ ]:
from sklearn.metrics import silhouette_score # Hint: this could be useful if you want to use silhouette_score as distance metric

In [ ]:
def cluster_with_no_k(word_to_paraphrases_dict):
    """
    Clusters paraphrases using any vector representation
    :param word_to_paraphrases_dict: dictionary, where key is a target word and value is a list of paraphrases
    :return: dictionary, where key is a target word and value is a list of list of paraphrases,
    where each list corresponds to a cluster
    """
    # Note: any vector representation should be in the same directory as this file
    vectors = Magnitude("GoogleNews-vectors-negative300.magnitude")
    clusterings = {}

    for target_word in word_to_paraphrases_dict.keys():
        paraphrase_list = word_to_paraphrases_dict[target_word]
        # TODO: Implement
        # hint: first find the best k value, you can define a seperate function, if needed.
        # hint: then fit a KMeans model on the data with k clusters
        clusterings[target_word] = None

    return clusterings

**Answer 3.4.1:** Run clustering on `dev` data, report the `f_scores` from the `dev` data [1 point]

**TODO:** [Report f_score on dev data]

In [ ]:
# YOUR CODE HERE (you can re-use reference code from 3.1)

As before, run the following command to generate the output file for the predicted clusterings for the test dataset.

In [ ]:
word_to_paraphrases_dict, _ = load_input_file('/content/test_nok_input.txt')
predicted_clusterings_nok = cluster_with_no_k(word_to_paraphrases_dict)
# write_to_output_file('test_output_nok.txt', predicted_clusterings_nok)

In [ ]:
# PennGrader - DO NOT CHANGE
# reload_grader()
grader.grade(test_case_id = 'test_q3_clusters_no_k', answer = (predicted_clusterings_nok, 'no k'))

**Answer 3.4.2:** Provide a brief description of your method in the report that includes the vectors you used, and any experimental results you have from running your model on the dev set.  [5 points]

**TODO**: [Describe your method]

# Submissions


## Leaderboards [2 points + 3 bonus]
In order to stir up some friendly competition, we would also like you to submit the clustering from your best model to a leaderboard. There will be 2 leaderboards to submit to:
- **Clusters with no K**: Copy the output file from your best model **(has to come from `3.4`)** to a file called `test_nok_output_leaderboard.txt` and include it with your submission in `HW5: Leaderboard Without K` following the format of the clusters file. [1 point]

- **Clusters with K**: Copy the output file from your best model **(has to come from `3.2 or 3.3`)** to a file called `test_output_leaderboard.txt` and include it with your submission in `HW5: Leaderboard With K` following the format of the clusters file. [1 point]

The first 10 places in either of the two leaderboards get 3 extra points.

## Free-response Checklist (check if you missed anything!)
We will look for the following free-responses in this notebook:
- Section 2: Question responses and analysis of correlations
- Section 3: For each clustering algorithm, you would need to report the `f_score` from the `dev` set and description of your methods.

## GradeScope File Submission
Here are the deliverables you need to submit to GradeScope:
- Write-up:
    - Part 2: answers to questions
    - Part 3: F-scores for clustering algorithms & discussions about your models
- Code:
    - This notebook and py file: rename to `homework5.ipynb` and `homework5.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`
- Leaderboard Results:
  - Leaderboard Without K: `test_nok_output_leaderboard.txt` (Task 3.4 output file)
  - Leaderboard With K: `test_output_leaderboard.txt` (Task 3.2 or 3.3 output file)